In [12]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [13]:
## Removing observations with NA
heart = heart.dropna()

In [14]:
## Defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'sysBP', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [15]:
## Scaling the data 
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [16]:
## Indentifying important variables with lasso
lasso_md = LassoCV().fit(X_train, Y_train)

## Extracting the best lambda
lasso_lambda = lasso_md.alpha_

## Fitting LASSO with best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.06631729,  0.2911493 ,  0.04417914,  0.06875482,  0.06832375,
       -0.01018952,  0.50360359,  0.46969805])

In [21]:
## Defining the input and target variables for logistic
X_train_logit = X_train[['age', 'BMI', 'totChol', 'sysBP', 'glucose']]
X_test_logit = X_test[['age', 'BMI', 'totChol', 'sysBP', 'glucose']]

## Building the logistic regression model 
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

## Predicting on test
logit_pred = logit_md.predict_proba(X_test_logit)[:, 1]

## Changing likelihoods to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, logit_label)

0.8979591836734694